# 4th Homework
## András Simon
---

Using OIDv4_ToolKit for downloading from Open Images Dateset

In [ ]:
!git clone https://github.com/EscVM/OIDv4_ToolKit.git

In [2]:
%cd OIDv4_ToolKit/

/content/OIDv4_ToolKit


In [ ]:
!pip3 install -r requirements.txt

We can download the files using the main.py. The desired class names and the data section (train, validation or test) can be adjusted.

In [ ]:
!python main.py downloader --classes Apple Orange Dog --type_csv train --limit 400

In [ ]:
!python main.py downloader --classes Apple Orange Dog --type_csv validation --limit 100

In [ ]:
!python main.py downloader --classes Apple Orange Dog --type_csv test --limit 100

Import some basic libraries

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import os

After the download, lets save the directory names for the ImageDataGenerator

In [17]:
base_dir = '/content/OIDv4_ToolKit/OID/Dataset/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

Import InceptionV3 and other used things form tensorflow.keras

In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K

Set the image width and heigth for the ImageDataGenerator

In [10]:
img_height=299
img_width=299

The following part is based on the CNN-transfer-learning-Keras.ipynb notebook from the practice

Create the model

In [11]:
base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output

x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

87916544/87910968 [==============================] - 0s 0us/step


Freeze the layers of the original InceptionV3 part of the model

In [12]:
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', metrics=['accuracy'],loss='categorical_crossentropy')

Using ImageDataGenerator for training 

In [13]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(img_height, img_width), batch_size=20, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(validation_dir, target_size=(img_height, img_width), batch_size=20, class_mode='categorical')

Found 1200 images belonging to 3 classes.
Found 207 images belonging to 3 classes.


Train only for 3 epoch

In [14]:
model.fit_generator(train_generator,
                    steps_per_epoch=30,
                    validation_data=validation_generator,
                    validation_steps=10,
                    epochs=3)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/3
30/30 [==============================] - 23s 761ms/step - loss: 0.8451 - accuracy: 0.7100 - val_loss: 0.1256 - val_accuracy: 0.9700
Epoch 2/3
30/30 [==============================] - 22s 736ms/step - loss: 0.2878 - accuracy: 0.8867 - val_loss: 0.0768 - val_accuracy: 0.9700
Epoch 3/3
30/30 [==============================] - 22s 740ms/step - loss: 0.3559 - accuracy: 0.8800 - val_loss: 0.1080 - val_accuracy: 0.9550


Now train the upper convolution layers too

In [14]:
for layer in model.layers[:172]:
       layer.trainable = False
for layer in model.layers[172:]:
       layer.trainable = True

model.compile(optimizer='adam', metrics=['accuracy'],loss='categorical_crossentropy')

In [15]:
model.fit_generator(train_generator,
                    steps_per_epoch=30,
                    validation_data=validation_generator,
                    validation_steps=10,
                    epochs=3)

Epoch 1/3
30/30 [==============================] - 22s 744ms/step - loss: 0.3581 - accuracy: 0.8700 - val_loss: 0.1129 - val_accuracy: 0.9650
Epoch 2/3
30/30 [==============================] - 22s 731ms/step - loss: 0.2319 - accuracy: 0.9117 - val_loss: 0.0854 - val_accuracy: 0.9600
Epoch 3/3
30/30 [==============================] - 22s 737ms/step - loss: 0.1643 - accuracy: 0.9333 - val_loss: 0.0744 - val_accuracy: 0.9750


Test with evaluate_generator

In [18]:
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(img_height, img_width), batch_size=20, class_mode='categorical')

Found 300 images belonging to 3 classes.


In [19]:
model.evaluate_generator(test_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


[0.17551380395889282, 0.9300000071525574]

In [20]:
model.metrics_names

['loss', 'accuracy']

The output of evaluate_generator is loss and accuracy